# Size a HPP plant based on a simplified hpp model

This **static** notebook illustrates how to solve hybrid power plant sizing optimization in a specific location based on preselected turbine type.

**To execute this setup a server is required as it relies on paralle evaluation of the model.**

Sizing a hybrid power plant consists on designing the following parameters:

### Design Variables

**Wind Plant design:**

- Number of wind turbines in the wind plant [-] (`Nwt`)
- Wind power installation density [MW/km2] (`wind_MW_per_km2`): This parameter controls how closely spaced are the turbines, which in turns affect how much wake losses are present.

**PV Plant design:**

- Solar plant power capacity [MW] (`solar_MW`)

**Battery Storage design:**

- Battery power [MW] (`b_P`)
- Battery energy capacity in hours [MWh] (`b_E_h `): Battery storage capacity in hours of full battery power (`b_E = b_E_h * b_P `). 
- Cost of battery power fluctuations in peak price ratio [-] (`cost_of_batt_degr`): This parameter controls how much penalty is given to do ramps in battery power in the HPP operation.


### Possible objective functions

- LCOE: Levelized cost of energy.
- IRR: Internal rate of return. It is not defined for projects that produce negative Net present values. Hydesigb returns IRR = 0 if the NPV < 0. Nevertheless, optimizations can be problematic for sites without a clear case. 
- NPV/CAPEX: Net present value over total CAPEX. A good proxy variable, that will produce optimal sites with the optimal IRR, but that is defined on sites with negative NPV.

The available variables for optimiaztion are:

```
 'NPV_over_CAPEX',
 'NPV [MEuro]',
 'IRR',
 'LCOE [Euro/MWh]',
 'CAPEX [MEuro]',
 'OPEX [MEuro]',
 'penalty lifetime [MEuro]',
 ```
 

In [1]:
# Install hydesign if needed
import importlib
if not importlib.util.find_spec("hydesign"):
    !pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git    

In [1]:
import pandas as pd
from hydesign.EGO_surrogate_based_optimization_P2X import EGO_path
from hydesign.examples import examples_filepath

# Optimize using EGO

In [2]:
EGO_P2X = f'{EGO_path}EGO_surrogate_based_optimization_P2X.py'

In [ ]:
%run $EGO_P2X \
    --example 9 \
    --opt_var "NPV_over_CAPEX"\
    --num_batteries 1\
    --n_procs 31\
    --n_doe 16\
    --n_clusters 4\
    --n_seed 0\
    --max_iter 1\
    --final_design_fn 'hydesign_P2X_output.csv'

Selected example site:
---------------------------------------------------
case                                          Europe
name                               Denmark_good_wind
longitude                                   8.594398
latitude                                   56.227322
altitude                                        85.0
input_ts_fn    Europe/input_ts_Denmark_good_wind.csv
sim_pars_fn                      Europe/hpp_pars.yml
price_fn                    Europe/2030-EL_PRICE.csv
price_col                                       DK_W
Name: 9, dtype: object




Sizing a HPP plant at Denmark_good_wind:

longitude = 8.594398
latitude = 56.227322
altitude = 85.0



